### This notebook is mostly based on the Mexico City Neighborhood:

#### Before we get the data and start exploring it, let's download all the dependencies that we will need

## Table of contents 

1. [Introduction: ](#introduction)
2. [Download and Explore Dataset](#download)
3. [Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them Mexico City](#foursquare)
4. [Analysis](#analysis)
5. [Explore Neighborhoods in Mexico City](#results)
6. [Analyze Each Neighborhood of Mexico City](#conclusion)
7. [Cluster Neighbourhoods](#neighbord) 
7. [Results](#clusters)

## 1.Introduction: <a name="introduction"></a>

In [1]:
import pandas as pd 
import requests 
import numpy as np
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim 
from sklearn.cluster import KMeans
import folium 
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
import wget
import json


import pandas as pd
from IPython.display import display
print('Libraries imported.')

Libraries imported.


In [2]:
#First, let's write a function to sort the venues in descending order
    
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]  

## 2.Download and Explore Dataset <a name="download"></a>

#### For convenience, I uploaded the file to github, so I only need to run a 'wget' command and access the data. So let's go ahead and do that.

In [3]:
df = pd.read_csv('D:\curso Coursera\Curso 9 Proyecto Final\Semana 5\Capstone Project The Battle of Neighborhood Part 2\cdmx_Nuevo.csv')

df = df.drop(columns=['ENTIDAD', 'CVE_ALC', 'CVE_COL','SECC_COM', 'SECC_PAR', 'Geo Shape'])
df['latitud'], df['longitud'] = df['Geo Point'].str.split(',', 1).str
df = df.drop(columns=["Geo Point"])
column_names = ['Borough', 'Neighbourhood', 'Latitude', 'Longitude'] 
df.columns = column_names
df = df[-df.isnull().any(axis=1)]

In [4]:
df.columns

Index(['Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

### Loading the json file from local address

In [5]:
url = 'D:\curso Coursera\Curso 9 Proyecto Final\Semana 5\Capstone Project The Battle of Neighborhood Part 2\ibmdatas5.json'

# 3.Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them Mexico City. <a name="foursquare"></a>

#### Define Foursquare Credentials and Version

In [6]:
address = 'México City, MX'

In [7]:
CLIENT_ID = 'WYAEZZVT00PMD0KJIHOFRGBAQXFI0UMRTDKQWGLKABOWBVL0' # your Foursquare ID
CLIENT_SECRET = 'EEMR5CG1PFXABI4PNF1CPGFNIBQ3VAO4U1LJ230KAPJZIV1L' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WYAEZZVT00PMD0KJIHOFRGBAQXFI0UMRTDKQWGLKABOWBVL0
CLIENT_SECRET:EEMR5CG1PFXABI4PNF1CPGFNIBQ3VAO4U1LJ230KAPJZIV1L


In [8]:
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mexico City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mexico City are 19.4326009, -99.1333416.


In [9]:
neighborhood_lat = df.loc[:,'Latitude'] 
neighborhood_long = df.loc[:,'Longitude'] 
neighborhood_name = df.loc[:,'Neighbourhood'] 

#### Loading the json file from local address

In [10]:
neighborhood_lat = 19.4326009 
neighborhood_long = -99.1333416 

#### Now, let's get the top 100 venues that are in Mexico City within a radius of 500 meters.

#### First, let's create the GET request URL. Name your URL url (Esta aprte fue omitida y el json se carga de manera local)

In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#    CLIENT_ID, 
#    CLIENT_SECRET, 
#    VERSION, 
#    neighborhood_lat, 
#    neighborhood_long, 
#    radius, 
#    LIMIT)
#url # display URL

## Saving the json file 

In [ ]:
results = requests.get(url).json()
with open('ibmdatas5.json','w') as outfile:
    json.dump(results,  outfile) 
   # ensure_ascii=False,indent=4

## Reading the json file

Define the json file 

In [ ]:
#df_js = pd.read_json(url)

In [11]:
#mv_data_json = requests.get('https://github.com/dc36205/Coursera-capstone-finalproject/blob/master/ibmdatas5.json')
!wget -q -O 'ibmdatas5.json' https://raw.githubusercontent.com/dc36205/Coursera-capstone-finalproject/blob/master/ibmdatas5.json

print('Data downloaded!') 

Data downloaded!


#### Send the GET request and examine the results

In [12]:
results = requests.get('https://github.com/dc36205/Coursera-capstone-finalproject/blob/master/ibmdatas5.json')

In [13]:
with open('ibmdatas5.json') as json_data:
    results = json.load(json_data)
#json.dumps(results, indent=4)
results

{'meta': {'code': 200, 'requestId': '5dda02231a4b0a001b8059dc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Centro',
  'headerFullLocation': 'Centro, Mexico City',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 179,
  'suggestedBounds': {'ne': {'lat': 19.437100904500006,
    'lng': -99.12857867011567},
   'sw': {'lat': 19.428100895499995, 'lng': -99.13810452988432}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b058700f964a520827a22e3',
       'name': 'Catedral Metropolitana de la Asunción de María',
       'location': {'address': 'Plaza de la Constitución S/N',
        'lat': 19.433526472529614,
        'lng': -99.13320365052626,
        'distance': 104,

In [138]:
results.keys()

dict_keys(['meta', 'response'])

In [166]:
for (k, v) in results.items():
   print("Key: " + k)
   print("Value: " + str(v))

Key: meta
Value: {'code': 200, 'requestId': '5dda02231a4b0a001b8059dc'}
Key: response
Value: {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Centro', 'headerFullLocation': 'Centro, Mexico City', 'headerLocationGranularity': 'neighborhood', 'totalResults': 179, 'suggestedBounds': {'ne': {'lat': 19.437100904500006, 'lng': -99.12857867011567}, 'sw': {'lat': 19.428100895499995, 'lng': -99.13810452988432}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b058700f964a520827a22e3', 'name': 'Catedral Metropolitana de la Asunción de María', 'location': {'address': 'Plaza de la Constitución S/N', 'lat': 19.433526472529614, 'lng': -99.13320365052626, 'distance': 104, 'postalCode': '06000', 'cc': 'MX', 'city': 'Cuauhtemoc', 'state': 'Distrito Federal

In [167]:
df_js = pd.read_json(url)

In [168]:
type(df_js)

pandas.core.frame.DataFrame

## 2. Transform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe.

In [170]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name'] 

In [ ]:
df_js.keys(['groups'][0]['items'])

In [163]:
venues = df_js['response']['groups'][0]['items']

In [172]:
type(venues)

list

In [145]:
type(df_js)

pandas.core.frame.DataFrame

In [131]:
# define the dataframe columns
column_names = ['Postalcode', 'Municipio', 'Categoria', 'Latitud', 'Longitud', 'Nombre'] 

In [164]:
nearby_venues = json_normalize(venues) #flatten JSON

In [133]:
nearby_venues['venue.name']

0        Catedral Metropolitana de la Asunción de María
1                                      Centro Histórico
2                                  Zócalo Central Hotel
3                           Gran Hotel Ciudad de México
4                                     Palacio de Hierro
                            ...                        
95                                   Plaza del Artesano
96    Banamex Palacio de los Condes de San Mateo de ...
97                                              Bukhara
98                     Antiguo Colegio de San Ildefonso
99                                       Centro Bolsero
Name: venue.name, Length: 100, dtype: object

In [21]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.postalCode','venue.location.city']
nearby_venues =nearby_venues.loc[:, filtered_columns]

In [22]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

In [23]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [24]:
nearby_venues.shape

(100, 6)

### Eliminando registros NaN

In [40]:
#nearby_venues['postalCode']
nearby_venues.dropna(subset=['postalCode'], how='all', inplace = True)

In [41]:
nearby_venues.head(15) 

,name,categories,lat,lng,postalCode,city
0,Catedral Metropolitana de la Asunción de María,Church,19.433526,-99.133204,06000,Cuauhtemoc
1,Centro Histórico,Plaza,19.433074,-99.135241,06010,Ciudad de México
2,Zócalo Central Hotel,Hotel,19.433919,-99.134395,6000,Ciudad de México
3,Gran Hotel Ciudad de México,Hotel,19.432137,-99.134468,06000,Cuauhtemoc
4,Palacio de Hierro,Department Store,19.431112,-99.134147,06060,Ciudad de México
5,Mercaderes,Restaurant,19.433979,-99.134825,06000,Ciudad de México
7,El Balcón del Zócalo,Mexican Restaurant,19.433852,-99.134142,06010,Cuahutémoc
9,Museo Mexicano Del Diseño (MUMEDI),Museum,19.433322,-99.134482,06000,Cuauhtemoc
10,Murales de Diego Rivera en la Secretaría de Ed...,Art Museum,19.432621,-99.131642,06010,Ciudad de México
11,El Cardenal,Mexican Restaurant,19.433746,-99.135216,06000,Ciudad de México


Then let's assign each dictionary to a list and create the dataframe.

In [129]:
#Initializing the lists
postalcode,borough,category,latitude,longitude, name = [],[],[],[],[],[]

#creating the Dataframe from a Dictionary forming columns from json data
#postalcode=list(nearby_venues['venue.location.postalCode']) 
postalcode = nearby_venues['postalCode']
borough = nearby_venues['city']
category = nearby_venues['categories']
latitude = nearby_venues['lat']
longitude = nearby_venues['lng']
name  = nearby_venues['name']

#borough =  list(nearby_venues['venue.location.city']) 
#category = list(nearby_venues['venue.categories'])
#latitude = list(nearby_venues['venue.location.lat'])
#longitude =list(nearby_venues['venue.location.lng'])  
#name = list(nearby_venues['venue.name'])  

# instantiate the dataframe
dict = {'Postalcode': postalcode, 'Municipio': borough, 'Categoria': category, 'Latitud': latitude, 'Longitud': longitude,
    'Nombre': name}

neighborhoods = pd.DataFrame(dict, columns=column_names)
neighborhoods.head(15)


,Postalcode,Municipio,Categoria,Latitud,Longitud,Nombre
0,06000,Cuauhtemoc,Church,19.433526,-99.133204,Catedral Metropolitana de la Asunción de María
1,06010,Ciudad de México,Plaza,19.433074,-99.135241,Centro Histórico
2,6000,Ciudad de México,Hotel,19.433919,-99.134395,Zócalo Central Hotel
3,06000,Cuauhtemoc,Hotel,19.432137,-99.134468,Gran Hotel Ciudad de México
4,06060,Ciudad de México,Department Store,19.431112,-99.134147,Palacio de Hierro
5,06000,Ciudad de México,Restaurant,19.433979,-99.134825,Mercaderes
7,06010,Cuahutémoc,Mexican Restaurant,19.433852,-99.134142,El Balcón del Zócalo
9,06000,Cuauhtemoc,Museum,19.433322,-99.134482,Museo Mexicano Del Diseño (MUMEDI)
10,06010,Ciudad de México,Art Museum,19.432621,-99.131642,Murales de Diego Rivera en la Secretaría de Ed...
11,06000,Ciudad de México,Mexican Restaurant,19.433746,-99.135216,El Cardenal


#### Now we are ready to clean the json.

In [51]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

73 venues were returned by Foursquare.


#### Structure the json file into a pandas dataframe.

In [52]:
neighborhoods.head(15)

,Postalcode,Municipio,Categoria,Latitud,Longitud,Nombre
0,06000,Cuauhtemoc,Church,19.433526,-99.133204,Catedral Metropolitana de la Asunción de María
1,06010,Ciudad de México,Plaza,19.433074,-99.135241,Centro Histórico
2,6000,Ciudad de México,Hotel,19.433919,-99.134395,Zócalo Central Hotel
3,06000,Cuauhtemoc,Hotel,19.432137,-99.134468,Gran Hotel Ciudad de México
4,06060,Ciudad de México,Department Store,19.431112,-99.134147,Palacio de Hierro
5,06000,Ciudad de México,Restaurant,19.433979,-99.134825,Mercaderes
7,06010,Cuahutémoc,Mexican Restaurant,19.433852,-99.134142,El Balcón del Zócalo
9,06000,Cuauhtemoc,Museum,19.433322,-99.134482,Museo Mexicano Del Diseño (MUMEDI)
10,06010,Ciudad de México,Art Museum,19.432621,-99.131642,Murales de Diego Rivera en la Secretaría de Ed...
11,06000,Ciudad de México,Mexican Restaurant,19.433746,-99.135216,El Cardenal


In [53]:
neighborhoods.shape

(73, 6)

In [128]:
neighborhoods.loc[0:]

,Postalcode,Municipio,Categoria,Latitud,Longitud,Nombre
0,06000,Cuauhtemoc,Church,19.433526,-99.133204,Catedral Metropolitana de la Asunción de María
1,06010,Ciudad de México,Plaza,19.433074,-99.135241,Centro Histórico
2,6000,Ciudad de México,Hotel,19.433919,-99.134395,Zócalo Central Hotel
3,06000,Cuauhtemoc,Hotel,19.432137,-99.134468,Gran Hotel Ciudad de México
4,06060,Ciudad de México,Department Store,19.431112,-99.134147,Palacio de Hierro
...,...,...,...,...,...,...
92,06010,Ciudad de México,Concert Hall,19.435760,-99.131088,Anfiteatro Simón Bolívar: Música de Cámara
93,06000,Cuauhtemoc,Coffee Shop,19.433615,-99.136788,Starbucks
96,06000,Downtown,Art Museum,19.432137,-99.136044,Banamex Palacio de los Condes de San Mateo de ...
98,06020,Cuauhtemoc,Art Museum,19.435829,-99.130900,Antiguo Colegio de San Ildefonso


#### Printing number of boroughs and neighborhoods

In [55]:
print('The dataframe has {} Municipios and {} neighborhoods.'.format(
        len(neighborhoods['Municipio'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 8 Municipios and 73 neighborhoods.


## 5.Explore Neighborhoods in Mexico City <a name="results"></a>

#### Let's create a function to repeat the same process to all the neighborhoods in Mexico City

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    q_zip = zip(names, latitudes, longitudes)

    for name, lat, lng in q_zip:
            
        # create the API request URL
    #     url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
     #        CLIENT_ID, 
      #       CLIENT_SECRET, 
      #       VERSION, 
      #       lat, 
      #       lng, 
      #       radius, 
      #       LIMIT)
            
        # make the GET request
        #results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venues])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
#    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *cdmx_venues*. This data is from the cdmx_Nuevo.csv

In [57]:
cdmx_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )  

In [58]:
#check the size of the resulting dataframe
cdmx_venues.shape

(180800, 7)

In [59]:
cdmx_venues.iloc[1:]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Centro Histórico,19.433074,-99.135241,Plaza
2,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Zócalo Central Hotel,19.433919,-99.134395,Hotel
3,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Gran Hotel Ciudad de México,19.432137,-99.134468,Hotel
4,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Palacio de Hierro,19.431112,-99.134147,Department Store
5,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Mercaderes,19.433979,-99.134825,Restaurant
...,...,...,...,...,...,...,...
180795,VENUSTIANO CARRANZA,19.4436366242,-99.1145123602,Plaza del Artesano,19.429772,-99.135689,Arts & Crafts Store
180796,VENUSTIANO CARRANZA,19.4436366242,-99.1145123602,Banamex Palacio de los Condes de San Mateo de ...,19.432137,-99.136044,Art Museum
180797,VENUSTIANO CARRANZA,19.4436366242,-99.1145123602,Bukhara,19.432397,-99.134080,Indian Restaurant
180798,VENUSTIANO CARRANZA,19.4436366242,-99.1145123602,Antiguo Colegio de San Ildefonso,19.435829,-99.130900,Art Museum


#### Let's check the size of the resulting dataframe

In [60]:
cdmx_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Catedral Metropolitana de la Asunción de María,19.433526,-99.133204,Church
1,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Centro Histórico,19.433074,-99.135241,Plaza
2,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Zócalo Central Hotel,19.433919,-99.134395,Hotel
3,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Gran Hotel Ciudad de México,19.432137,-99.134468,Hotel
4,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Palacio de Hierro,19.431112,-99.134147,Department Store
5,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Mercaderes,19.433979,-99.134825,Restaurant
6,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Moyo,19.433184,-99.134379,Frozen Yogurt Shop
7,MIGUEL HIDALGO,19.4016815485,-99.2354719599,El Balcón del Zócalo,19.433852,-99.134142,Mexican Restaurant
8,MIGUEL HIDALGO,19.4016815485,-99.2354719599,MUMEDI Café & Shop,19.433237,-99.134543,Café
9,MIGUEL HIDALGO,19.4016815485,-99.2354719599,Museo Mexicano Del Diseño (MUMEDI),19.433322,-99.134482,Museum


In [61]:
cdmx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ALVARO OBREGON,24900,24900,24900,24900,24900,24900
AZCAPOTZALCO,11100,11100,11100,11100,11100,11100
BENITO JUAREZ,6400,6400,6400,6400,6400,6400
COYOACAN,15300,15300,15300,15300,15300,15300
CUAJIMALPA DE MORELOS,4300,4300,4300,4300,4300,4300
CUAUHTEMOC,6300,6300,6300,6300,6300,6300
GUSTAVO A. MADERO,23200,23200,23200,23200,23200,23200
IZTACALCO,5500,5500,5500,5500,5500,5500
IZTAPALAPA,29300,29300,29300,29300,29300,29300


In [173]:
print('There are {} uniques categories.'.format(len(cdmx_venues['Venue Category'].unique())))

There are 49 uniques categories.


In [216]:
#1rst Cluster
cdmx_merged.loc[cdmx_merged['Cluster Labels'] == 0, cdmx_merged.columns[[1] + list(range(5, cdmx_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,MIGUEL HIDALGO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant,Museum,Coffee Shop,Department Store,Hotel,Ice Cream Shop
1,MIGUEL HIDALGO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant,Museum,Coffee Shop,Department Store,Hotel,Ice Cream Shop
2,MIGUEL HIDALGO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant,Museum,Coffee Shop,Department Store,Hotel,Ice Cream Shop
3,MIGUEL HIDALGO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant,Museum,Coffee Shop,Department Store,Hotel,Ice Cream Shop
4,MIGUEL HIDALGO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant,Museum,Coffee Shop,Department Store,Hotel,Ice Cream Shop
...,...,...,...,...,...,...,...,...,...,...,...
1803,COYOACAN,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant,Museum,Coffee Shop,Department Store,Hotel,Ice Cream Shop
1804,VENUSTIANO CARRANZA,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant,Museum,Coffee Shop,Department Store,Hotel,Ice Cream Shop
1805,VENUSTIANO CARRANZA,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant,Museum,Coffee Shop,Department Store,Hotel,Ice Cream Shop
1806,VENUSTIANO CARRANZA,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant,Museum,Coffee Shop,Department Store,Hotel,Ice Cream Shop


#### Let's find out how many unique categories can be curated from all the returned venues 

## 6.Analyze Each Neighborhood of Mexico City <a name="conclusion"></a>

In [251]:
#Analyse Each Neighbourhood   linea con el drop
#cdmx_onehot = pd.get_dummies(cdmx_venues[['Venue Category']], prefix="", prefix_sep="").drop('Neighborhood', axis=1)
cdmx_onehot = pd.get_dummies(cdmx_venues[['Venue Category']], prefix="", prefix_sep="")

In [252]:
# add neighborhood column back to dataframe
cdmx_onehot['Neighborhood'] = cdmx_venues['Neighborhood'] 
#cdmx_onehot.insert(0, 'Neighborhood', cdmx_onehot['Neighborhood'])esto lo hacen en el Lab de Toronto 

In [221]:
# move neighborhood column to the first column
#fixed_columns = [cdmx_onehot.columns[-1]] + list(cdmx_onehot.columns[:-1])
#cdmx_onehot = cdmx_onehot[fixed_columns]

In [253]:
cdmx_onehot.head()

,Art Museum,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Boutique,Brewery,...,Optical Shop,Plaza,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,Spanish Restaurant,Sporting Goods Shop,Taco Place,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MIGUEL HIDALGO
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,MIGUEL HIDALGO
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MIGUEL HIDALGO
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MIGUEL HIDALGO
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MIGUEL HIDALGO


In [254]:
print(len(cdmx_onehot[(cdmx_onehot.Neighborhood=="ALVARO OBREGON")]))
print(len(cdmx_onehot[(cdmx_onehot.Neighborhood=="ALVARO OBREGON")&(cdmx_onehot["Historic Site"]==1)]))

24900
498


In [255]:
cdmx_onehot.columns

Index(['Art Museum', 'Arts & Crafts Store', 'BBQ Joint', 'Bakery', 'Bar',
       'Baseball Field', 'Bed & Breakfast', 'Beer Garden', 'Boutique',
       'Brewery', 'Buffet', 'Café', 'Campground', 'Candy Store',
       'Chocolate Shop', 'Church', 'Clothing Store', 'Cocktail Bar',
       'Coffee Shop', 'Concert Hall', 'Cosmetics Shop', 'Deli / Bodega',
       'Department Store', 'Food', 'Frozen Yogurt Shop',
       'General College & University', 'General Entertainment',
       'Historic Site', 'History Museum', 'Hostel', 'Hotel', 'Ice Cream Shop',
       'Indian Restaurant', 'Indie Theater', 'Jewelry Store', 'Lounge',
       'Mexican Restaurant', 'Museum', 'Music Venue', 'Non-Profit',
       'Optical Shop', 'Plaza', 'Restaurant', 'Sandwich Place',
       'Seafood Restaurant', 'Snack Place', 'Spanish Restaurant',
       'Sporting Goods Shop', 'Taco Place', 'Neighborhood'],
      dtype='object')

In [256]:
#Let´s print the new size
print(cdmx_onehot.shape)

(180800, 50)


In [209]:
cdmx_onehot.to_csv('example.csv')

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [257]:
cdmx_grouped = cdmx_onehot.groupby('Neighborhood').mean().reset_index()
#print('grouping vecin', cdmx_grouped)
cdmx_grouped

,Neighborhood,Art Museum,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Boutique,...,Non-Profit,Optical Shop,Plaza,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,Spanish Restaurant,Sporting Goods Shop,Taco Place
0,ALVARO OBREGON,0.05,0.05,0.01,0.06,0.03,0.01,0.01,0.01,0.02,...,0.01,0.01,0.01,0.07,0.02,0.02,0.01,0.01,0.01,0.02
1,AZCAPOTZALCO,0.05,0.05,0.01,0.06,0.03,0.01,0.01,0.01,0.02,...,0.01,0.01,0.01,0.07,0.02,0.02,0.01,0.01,0.01,0.02
2,BENITO JUAREZ,0.05,0.05,0.01,0.06,0.03,0.01,0.01,0.01,0.02,...,0.01,0.01,0.01,0.07,0.02,0.02,0.01,0.01,0.01,0.02
3,COYOACAN,0.05,0.05,0.01,0.06,0.03,0.01,0.01,0.01,0.02,...,0.01,0.01,0.01,0.07,0.02,0.02,0.01,0.01,0.01,0.02
4,CUAJIMALPA DE MORELOS,0.05,0.05,0.01,0.06,0.03,0.01,0.01,0.01,0.02,...,0.01,0.01,0.01,0.07,0.02,0.02,0.01,0.01,0.01,0.02
5,CUAUHTEMOC,0.05,0.05,0.01,0.06,0.03,0.01,0.01,0.01,0.02,...,0.01,0.01,0.01,0.07,0.02,0.02,0.01,0.01,0.01,0.02
6,GUSTAVO A. MADERO,0.05,0.05,0.01,0.06,0.03,0.01,0.01,0.01,0.02,...,0.01,0.01,0.01,0.07,0.02,0.02,0.01,0.01,0.01,0.02
7,IZTACALCO,0.05,0.05,0.01,0.06,0.03,0.01,0.01,0.01,0.02,...,0.01,0.01,0.01,0.07,0.02,0.02,0.01,0.01,0.01,0.02
8,IZTAPALAPA,0.05,0.05,0.01,0.06,0.03,0.01,0.01,0.01,0.02,...,0.01,0.01,0.01,0.07,0.02,0.02,0.01,0.01,0.01,0.02
9,LA MAGDALENA CONTRERAS,0.05,0.05,0.01,0.06,0.03,0.01,0.01,0.01,0.02,...,0.01,0.01,0.01,0.07,0.02,0.02,0.01,0.01,0.01,0.02


In [179]:
cdmx_grouped1 = cdmx_onehot.groupby(['Neighborhood','Restaurant']).count() #.reset_index()
#print('grouping vecin', cdmx_grouped)
cdmx_grouped1

Art Museum  Arts & Crafts Store  BBQ Joint  \
Neighborhood           Restaurant                                               
ALVARO OBREGON         0                23157                23157      23157   
                       1                 1743                 1743       1743   
AZCAPOTZALCO           0                10323                10323      10323   
                       1                  777                  777        777   
BENITO JUAREZ          0                 5952                 5952       5952   
                       1                  448                  448        448   
COYOACAN               0                14229                14229      14229   
                       1                 1071                 1071       1071   
CUAJIMALPA DE MORELOS  0                 3999                 3999       3999   
                       1                  301                  301        301   
CUAUHTEMOC             0                 5859                 5859       5859   
                       1                  441                  441        441   
GUSTAVO A. MADERO      0                21576                21576      21576   
                       1                 1624                 1624       1624   
IZTACALCO              0                 5115                 5115       5115   
                       1                  385                  385        385   
IZTAPALAPA             0                27249                27249      27249   
                       1                 2051                 2051       2051   
LA MAGDALENA CONTRERAS 0                 4836                 4836       4836   
                       1                  364                  364        364   
MIGUEL HIDALGO         0                 8184                 8184       8184   
                       1                  616                  616        616   
MILPA ALTA             0                 1023                 1023       1023   
                       1                   77                   77         77   
TLAHUAC                0                 5394                 5394       5394   
                       1                  406                  406        406   
TLALPAN                0                16461                16461      16461   
                       1                 1239                 1239       1239   
VENUSTIANO CARRANZA    0                 7440                 7440       7440   
                       1                  560                  560        560   
XOCHIMILCO             0                 7347                 7347       7347   
                       1                  553                  553        553   

                                   Bakery    Bar  Baseball Field  \
Neighborhood           Restaurant                                  
ALVARO OBREGON         0            23157  23157           23157   
                       1             1743   1743            1743   
AZCAPOTZALCO           0            10323  10323           10323   
                       1              777    777             777   
BENITO JUAREZ          0             5952   5952            5952   
                       1              448    448             448   
COYOACAN               0            14229  14229           14229   
                       1             1071   1071            1071   
CUAJIMALPA DE MORELOS  0             3999   3999            3999   
                       1              301    301             301   
CUAUHTEMOC             0             5859   5859            5859   
                       1              441    441             441   
GUSTAVO A. MADERO      0            21576  21576           21576   
                       1             1624   1624            1624   
IZTACALCO              0             5115   5115            5115   
                       1              385    385             385   
IZTAPALAPA             0            27249  27249        

In [258]:
#Print each neighbourhood along with the top 5 most common venues
num_top_venues = 5

In [259]:
for hood in cdmx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = cdmx_grouped[cdmx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALVARO OBREGON----
                 venue  freq
0           Restaurant  0.07
1               Bakery  0.06
2           Art Museum  0.05
3   Mexican Restaurant  0.05
4  Arts & Crafts Store  0.05


----AZCAPOTZALCO----
                 venue  freq
0           Restaurant  0.07
1               Bakery  0.06
2           Art Museum  0.05
3   Mexican Restaurant  0.05
4  Arts & Crafts Store  0.05


----BENITO JUAREZ----
                 venue  freq
0           Restaurant  0.07
1               Bakery  0.06
2           Art Museum  0.05
3   Mexican Restaurant  0.05
4  Arts & Crafts Store  0.05


----COYOACAN----
                 venue  freq
0           Restaurant  0.07
1               Bakery  0.06
2           Art Museum  0.05
3   Mexican Restaurant  0.05
4  Arts & Crafts Store  0.05


----CUAJIMALPA DE MORELOS----
                 venue  freq
0           Restaurant  0.07
1               Bakery  0.06
2           Art Museum  0.05
3   Mexican Restaurant  0.05
4  Arts & Crafts Store  0.05


----CUA

In [93]:
#Let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

In [94]:
indicators = ['st', 'nd', 'rd']

In [260]:
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [261]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)

In [262]:
neighborhoods_venues_sorted 

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


In [263]:
neighborhoods_venues_sorted['Neighborhood'] = cdmx_grouped['Neighborhood']
neighborhoods_venues_sorted
for ind in np.arange(cdmx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cdmx_grouped.iloc[ind, :], num_top_venues)

In [264]:
neighborhoods_venues_sorted.head(7)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,ALVARO OBREGON,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
1,AZCAPOTZALCO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
2,BENITO JUAREZ,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
3,COYOACAN,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
4,CUAJIMALPA DE MORELOS,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
5,CUAUHTEMOC,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
6,GUSTAVO A. MADERO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant


In [265]:
neighborhoods_venues_sorted.columns

Index(['Neighborhood', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue'],
      dtype='object')

In [266]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,ALVARO OBREGON,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
1,AZCAPOTZALCO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
2,BENITO JUAREZ,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
3,COYOACAN,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
4,CUAJIMALPA DE MORELOS,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant


In [107]:
#type(neighborhoods_venues_sorted)
neighborhoods_venues_sorted.to_csv('example.csv')

## 7.Cluster Neighbourhoods <a name="neighbord"></a>

Run k-means to cluster the neighborhood into 5 clusters.

In [296]:
# set number of clusters
kclusters = 5

cdmx_grouped_clustering = cdmx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cdmx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:969: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [269]:
#Add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [284]:
cdmx_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cdmx_merged  = cdmx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
#cdmx_merged = cdmx_merged_merged.reset_index(drop=True)
cdmx_merged = cdmx_merged.reset_index(drop=True)

In [286]:
#neighborhoods_venues_sorted.head(5)
neighborhoods_venues_sorted.to_csv('neighborhoods_venues_sorted4.csv')

In [272]:
cdmx_merged

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,LOMAS DE REFORMA,MIGUEL HIDALGO,19.4016815485,-99.2354719599,0,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
1,DANIEL GARZA (AMPL),MIGUEL HIDALGO,19.4092184712,-99.1938392453,0,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
2,IGNACIO MANUEL ALTAMIRANO,MIGUEL HIDALGO,19.4631440514,-99.1968280035,0,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
3,LEGARIA,MIGUEL HIDALGO,19.4555305454,-99.1930482732,0,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
4,LEGARIA (U HAB),MIGUEL HIDALGO,19.450020036,-99.2010760751,0,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
...,...,...,...,...,...,...,...,...,...,...
1803,ADOLFO RUIZ CORTINES II,COYOACAN,19.31784374,-99.1550720386,0,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
1804,EL ARENAL 1A SECCION,VENUSTIANO CARRANZA,19.4250609099,-99.0633202592,0,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
1805,AERONAUTICA MILITAR,VENUSTIANO CARRANZA,19.4208650922,-99.1157518113,0,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
1806,IGNACIO ZARAGOZA II,VENUSTIANO CARRANZA,19.4117785299,-99.087917303,0,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant


In [190]:
cdmx_merged.to_csv('cdmx_merged.csv')

In [ ]:
#Finally, let's visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

In [275]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
#rainbow = ['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']
rainbow = ['#00b5eb', '#80ffb4', '#ffb360', '#ff0000','#8000ff']

In [ ]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cdmx_merged['Latitude'], cdmx_merged['Longitude'], cdmx_merged['Neighbourhood'], cdmx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [240]:
map_clusters.save("map_clusters_mexicocityNew4.html ") 

In [241]:
map_clusters

## 8.Results <a name="clusters"></a>

### Cluster 1

In [276]:
#1rst Cluster
cdmx_merged.loc[cdmx_merged['Cluster Labels'] == 0, cdmx_merged.columns[[1] + list(range(5, cdmx_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,MIGUEL HIDALGO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
1,MIGUEL HIDALGO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
2,MIGUEL HIDALGO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
3,MIGUEL HIDALGO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
4,MIGUEL HIDALGO,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
...,...,...,...,...,...,...
1803,COYOACAN,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
1804,VENUSTIANO CARRANZA,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
1805,VENUSTIANO CARRANZA,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant
1806,VENUSTIANO CARRANZA,Restaurant,Bakery,Art Museum,Arts & Crafts Store,Mexican Restaurant


### Cluster 2

In [277]:
cdmx_merged.loc[cdmx_merged['Cluster Labels'] == 1, cdmx_merged.columns[[1] + list(range(5, cdmx_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


### Cluster 3 

In [278]:
#C3
cdmx_merged.loc[cdmx_merged['Cluster Labels'] == 2, cdmx_merged.columns[[1] + list(range(5, cdmx_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


### Cluster 4

In [279]:
#C4
cdmx_merged.loc[cdmx_merged['Cluster Labels'] == 3, cdmx_merged.columns[[1] + list(range(5, cdmx_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


### Cluster 5

In [280]:
#C5
cdmx_merged.loc[cdmx_merged['Cluster Labels'] == 4, cdmx_merged.columns[[1] + list(range(5, cdmx_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
